In [1]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import random
import time

class Game:
    def __init__(self, root):
        self.root = root
        self.sessions = []
        self.total_attempted = 0
        self.total_correct = 0
        self.current_session_index = 0
        self.current_session = None
        self.player_name = ""
        self.game_results = pd.DataFrame(columns=['Player', 'Total Questions', 'Accuracy'])

        self.create_widgets()
    
    def create_widgets(self):
        self.label = tk.Label(self.root, text="Numerical Efficiency Test", font=("Helvetica", 16))
        self.label.pack(pady=20)
        
        self.name_label = tk.Label(self.root, text="Enter Player Name:", font=("Helvetica", 14))
        self.name_label.pack(pady=10)
        
        self.name_entry = tk.Entry(self.root, font=("Helvetica", 14))
        self.name_entry.pack(pady=10)
        
        self.start_button = tk.Button(self.root, text="Start Game", command=self.start)
        self.start_button.pack(pady=10)
        
        self.question_label = tk.Label(self.root, text="", font=("Helvetica", 14))
        self.question_label.pack(pady=20)
        
        self.answer_entry = tk.Entry(self.root, font=("Helvetica", 14))
        self.answer_entry.pack(pady=10)
        
        self.submit_button = tk.Button(self.root, text="Submit Answer", command=self.submit_answer)
        self.submit_button.pack(pady=10)
        
        self.result_label = tk.Label(self.root, text="", font=("Helvetica", 14))
        self.result_label.pack(pady=20)
    
    def start(self):
        self.player_name = self.name_entry.get()
        if not self.player_name:
            messagebox.showwarning("Input Error", "Please enter the player's name.")
            return
        
        self.sessions = [Session(1, QuestionSpelling), Session(2, QuestionStars), Session(2, QuestionMath)]
        self.current_session_index = 0
        self.total_attempted = 0
        self.total_correct = 0
        self.next_session()
    
    def next_session(self):
        if self.current_session_index < len(self.sessions):
            self.current_session = self.sessions[self.current_session_index]
            self.current_session.start_session(self)
            self.current_session_index += 1
        else:
            self.generate_summary()
    
    def submit_answer(self):
        self.current_session.submit_answer(self)
    
    def generate_summary(self):
        accuracy = (self.total_correct / self.total_attempted) * 100 if self.total_attempted > 0 else 0
        summary = f"Summary Report:\nTotal Questions Attempted: {self.total_attempted}\nTotal Correct Answers: {self.total_correct}\nAccuracy: {accuracy:.2f}%\n"
        
        if accuracy > 90:
            summary += "Performance: Brilliant"
        elif accuracy > 75:
            summary += "Performance: Good"
        elif accuracy > 50:
            summary += "Performance: Need Attention"
        else:
            summary += "Performance: Need Special Attention"
        
        messagebox.showinfo("Summary", summary)
        
        self.update_game_results(self.player_name, self.total_attempted, accuracy)
    
    def update_game_results(self, player_name, total_questions, accuracy):
        new_result = pd.DataFrame([[player_name, total_questions, accuracy]], columns=['Player', 'Total Questions', 'Accuracy'])
        self.game_results = pd.concat([new_result, self.game_results]).head(50).reset_index(drop=True)
        print(self.game_results)  # Print the DataFrame to the console for debugging purposes

class Session:
    def __init__(self, duration, question_type):
        self.duration = duration * 60
        self.question_type = question_type
        self.questions = []
        self.questions_attempted = 0
        self.questions_correct = 0
        self.start_time = None
        self.current_question = None
    
    def start_session(self, game):
        self.start_time = time.time()
        self.ask_question(game)
    
    def ask_question(self, game):
        if time.time() - self.start_time < self.duration:
            self.current_question = self.question_type()
            game.question_label.config(text=self.current_question.question_text)
            game.answer_entry.delete(0, tk.END)
            game.result_label.config(text="")
        else:
            game.total_attempted += self.questions_attempted
            game.total_correct += self.questions_correct
            game.next_session()
    
    def submit_answer(self, game):
        user_answer = game.answer_entry.get()
        self.current_question.user_answer = user_answer
        self.questions_attempted += 1
        if self.current_question.is_correct():
            self.questions_correct += 1
            game.result_label.config(text="Correct!", fg="green")
        else:
            game.result_label.config(text=f"Wrong! The correct answer was {self.current_question.correct_answer}", fg="red")
        
        self.ask_question(game)

class Question:
    def __init__(self):
        self.question_text = ""
        self.correct_answer = ""
        self.user_answer = ""
    
    def is_correct(self):
        return self.user_answer.lower() == self.correct_answer.lower()

class QuestionSpelling(Question):
    def __init__(self):
        super().__init__()
        number = random.randint(0, 9)
        self.question_text = f"Spell the number {number}: "
        self.correct_answer = self.number_to_words(number)
    
    def number_to_words(self, number):
        words = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
        return words[number]

class QuestionStars(Question):
    def __init__(self):
        super().__init__()
        stars = random.randint(1, 9)
        self.question_text = f"Count the stars: {'*' * stars} "
        self.correct_answer = str(stars)

class QuestionMath(Question):
    def __init__(self):
        super().__init__()
        num1 = random.randint(0, 9)
        num2 = random.randint(0, 9)
        operation = random.choice(['+', '-'])
        self.question_text = f"What is {num1} {operation} {num2}? "
        self.correct_answer = str(eval(f"{num1} {operation} {num2}"))

# Create the main window
root = tk.Tk()
root.title("Numerical Efficiency Test")
game = Game(root)
root.mainloop()


    Player Total Questions   Accuracy
0  sudipto              78  91.025641
